In [ ]:
from music21 import instrument as m21instrument
from music21.meter.base import TimeSignature as m21ts
from music21.key  import KeySignature as m21ks
import pandas as pd
from pandas import DataFrame
from torch.nn import Module
from torch import tensor
import torch
import numpy as np
from EPMS import SerializationSettings, createPerformanceHeader

class PapagaioAgent:
    def __init__(self, predictor: Module,
                 # models_path: str,
                 settings: SerializationSettings):

        # predictor is the LSTM class with
        # its layers, activation functions, etc.
        #
        # ~control~
        self.predictor = predictor

        # models_path is the path to the directory
        # where the predictor's memory will be
        # stored at/retreived from
        #
        # ~predictors memories~
        # self.models_path = models_path

		# resolution, keyboard_size, keyboard_offset
        self.settings = settings

        # holds the information about the current beat
        # self.current_data = {}


    def serialDecode(self, serial: pd.DataFrame) -> (str,
                                               m21instrument,
                                               int,
                                               str,
                                               int,
                                               int,
                                               DataFrame,
                                               int,
                                               tensor,
                                               tensor):
        '''
            Receive a PMS serial of a SINGLE intrument and separate its blocks/contents.
            The serial must come with RESOLUTION rows.
        '''

        if len(serial) != self.settings.resolution:
            raise SyntaxError('Inconsistent serial size (doesnt match resolution).')

        source_name = list(set(serial.index)) # List
        source_instrument = list(set(serial.INSTRUMENT)) # List
        midi_program = list(set(serial.MIDI_PROGRAM)) # List
        sound = list(set(serial.SOUND)) # List

        # Verifying data consistency
        if  (len(source_name) > 1 or
            len(source_instrument) > 1 or
            len(midi_program) > 1 or
            len(sound) > 1):
            raise SyntaxError('Inconsistent Instrument Block.')

        # self.source_name = source_name[0]
        # self.source_instrument = source_instrument[0]
        # self.midi_program = midi_program[0]
        # self.sound = sound[0]
        source_name = source_name[0]
        source_instrument = source_instrument[0]
        midi_program = midi_program[0]
        sound = sound[0]


        measures = list(set(serial.MEASURE))
        beats = list(set(serial.BEAT))
        frames = list(set(serial.FRAME))

        if  (len(measures) > 1 or
            len(beats) > 1):
            raise SyntaxError('Inconsistent Metric Block.')

        # self.measure = measures[0]
        # self.beat = beats[0]
        measure = measures[0]
        beat = beats[0]

        # self.environment = serial[['ORIGINAL_KS', 'TS', 'TEMPO']].reset_index(drop=True)
        environment = serial[['ORIGINAL_KS', 'TS', 'TEMPO']].reset_index(drop=True)

        # self.ts_numerator = serial.loc('ORIGINAL_KS')[0][0]
        ts_numerator = serial.loc('ORIGINAL_KS')[0][0]

        performance = serial.iloc[:, (len(serial) - self.settings.keyboard_size - 1):]
        # self.mhe, self.velocities = performanceDecode(performance)
        mhe, velocities = performanceDecode(performance)

        # print(self.mhe.tolist())
        # print(self.velocities.tolist())
        print(ts_numerator)

        return (source_name,
                source_instrument,
                midi_program,
                sound,
                measure,
                beat,
                environment,
                ts_numerator,
                mhe,
                velocities)


    def performanceDecode(self, performance: pd.Dataframe) -> (list[int], list[int]):
        '''
            Receive a EPMS Performance Block and separe
            the data into a Multi Hot Encoding (MHE) representation
            of the notes played and a int (vector?) with
            the velocity of the notes (range 0-127)
        '''
        performance = performance.to_numpy()

        mhe = np.zeros(performance.shape).astype(int)
        velocities = np.zeros(performance.shape).astype(int)

        for i, frame in enumerate(performance):
            # print(frame)
            for j, note_value in enumerate(frame):
                if type(note_value) is bool:
                    mhe[i][j] = 0
                    velocities[i][j] = 0
                else:
                    mhe[i][j] = 1
                    velocities[i][j] = note_value

        return (torch.from_numpy(mhe), torch.from_numpy(velocities))


    def fetchMemory(self, source_instrument: m21instrument,
                    target_instrument: m21instrument):

        '''
            Access the models_path directory and search
            for a model trained accordingly.
            If success, return it. (or just set self.predictor??)
            Otherwise raise an error.
        '''
        pass


    def performanceEncode(self, mhe: np.array,
                          velocities: np.array) -> DataFrame:
        '''
            Receive the output of the predictor and
            the velocity value/vector.
            Return a PerformanceBlock, a pandas DataFrame
        '''

        # precisamos implementar algumas funcoes do EPMS
        # para facilitar essa parte

        performance_header = createPerformanceHeader(self.settings.keyboard_size)

        performance_data = mhe * velocities

        # zeros to false
        for frame in performance_data:
			for note in frame:
				if note == 0:
				    note = False

        performance_block = pd.DataFrame(performance_data, columns=performance_header)

        return performance_block


    def serialEncode(self, source_name: str,
                     target_instrument: m21instrument,
                     midi_program: int,
                     sound: str,
                     measure: int,
                     beat: int,
                     environment_block: DataFrame,
                     ts_numerator: int,
                     performance_block: DataFrame) -> DataFrame:
        '''
            Receive all information needed to create a
            PMS serial, including generated info such as
            the performance block and its velocities.
        '''

		# check if should increase measure number
		if beat == ts_numerator:
			beat = 1
			measure += 1

        pass


    def feedContext(self, input_serial: DataFrame) -> None:
		'''
			Receive a serial with N beats.
			Iterate over them, feeding the NN.
			Here we dont care about the predicted values,
			our objective is just to update the model
			hidden and cell states.
		'''

		performance = input_serial.iloc[:, (len(input_serial) - self.settings.keyboard_size - 1):]
		mhe, velocities = performanceDecode(performance)



		for beat in input_serial:
			_, (hidden, cell) = self.predictor(mhe, hidden, cell)

		return


    def generate(input_serial: Union[DataFrame, None],
                 target_instrument: m21instrument,
                 beat_amount: int=1,
                 temperature: float=0.5) -> DataFrame:
        '''
            Receive a serial containing a SINGLE instrument
            and return a serial generated by the predictor
            having input_serial as context
        '''
        pass

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-12-821ad35215de>, line 169)